#  Agente Conversacional

## Autores
- ### Luis Benavides
- ### Juan Jiménez
- ### Alex Naranjo

## Configuraciones

In [ ]:
import os
import sys
import torch
import re
import unicodedata
import pdfplumber
import pandas as pd
from nltk.tokenize import sent_tokenize
import nltk
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import langchain
import pkgutil
import glob
import json
import numpy as np
from typing import List, Tuple

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from duckduckgo_search import DDGS
from langchain_core.tools import Tool

from langchain_community.utilities import SerpAPIWrapper

from langchain_ollama import OllamaLLM
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

import spacy

import streamlit as st

from pdfminer.high_level import extract_text
import unicodedata, re

# NLTK 3.9+ requiere 'punkt_tab' además de 'punkt'. Descargamos ambos de forma silenciosa.
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)

# Verificar GPU disponible
if torch.cuda.is_available():
	print("Usando GPU:", torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\lfben\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\metrics\association.py:26: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.4)
  from scipy.stats import fisher_exact
c:\Users\lfben\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Usando GPU: NVIDIA GeForce RTX 2060


## 1. Base de Datos

In [2]:
carpeta_data = "data"
carpeta_pdf = "data/pdf"
carpeta_processed = "data/processed/clean_text"
carpeta_embeddings = "data/embeddings"
carpeta_chunks_fijo = "data/processed/chunks_fixed"
carpeta_chunks_sem = "data/processed/chunks_semantic"
carpeta_db = "data/databases"

agent_name = "QWERTY"

# Crear carpetas necesarias
for folder in [carpeta_data, carpeta_pdf, carpeta_processed, carpeta_embeddings, carpeta_chunks_sem, carpeta_chunks_fijo, carpeta_db]:
	os.makedirs(folder, exist_ok=True)
print("Estructura de carpetas creada correctamente.")

Estructura de carpetas creada correctamente.


## 2. Preprocesamiento Textual

En esta etapa el objetivo es convertir los PDFs de los apuntes en texto limpio y homogéneo, eliminando caracteres extraños, tildes mal codificadas, saltos de línea innecesarios y normalizando todo a minúsculas.

Esto permitirá que los embeddings sean más consistentes y que la base vectorial pueda recuperar mejor los fragmentos de texto relevantes.

In [ ]:
# === 1. Extracción ===
def extraer_texto_pdf(ruta_pdf: str) -> str:
	"""
		Extrae texto del PDF usando pdfminer y corrige pseudoacentos ASCII
	"""
	texto = extract_text(ruta_pdf) or ""
	texto = unicodedata.normalize("NFC", texto)

	# --- Correcciones comunes para PDFs en español mal codificados ---
	patrones = {
		r"˜n": "ñ",
		r"˜N": "Ñ",
		r" ´A": "Á",
		r" ´E": "É",
		r" ´I": "Í",
		r" ´O": "Ó",
		r" ´U": "Ú",
		r"´A": "Á",
		r"´E": "É",
		r"´I": "Í",
		r"´O": "Ó",
		r"´U": "Ú",
		r"´ı": "í",
		r"´a": "á",
		r"´e": "é",
		r"´i": "í",
		r"´o": "ó",
		r"´u": "ú",
		r"´n": "ñ",
		r"ı´": "í",
		r"a´": "á",
		r"e´": "é",
		r"i´": "í",
		r"o´": "ó",
		r"u´": "ú",
		r"A´": "Á",
		r"E´": "É",
		r"I´": "Í",
		r"O´": "Ó",
		r"U´": "Ú",
	}

	for k, v in patrones.items():
		texto = re.sub(k, v, texto)

	# --- Limpieza básica adicional ---
	texto = unicodedata.normalize("NFC", texto)
	texto = re.sub(r"[ \t]+", " ", texto)     # espacios dobles
	texto = re.sub(r"\n{3,}", "\n\n", texto)  # saltos excesivos
	texto = texto.strip()

	return texto


In [ ]:
def limpiar_texto(texto: str) -> str:
	"""
		Limpia y normaliza el texto extraído de PDFs.
		- Convierte a minúsculas.
		- Elimina tildes (á→a, ñ→n, etc.).
		- Quita caracteres especiales o símbolos raros.
		- Normaliza espacios.
	"""

	if not texto:
		return ""

	# --- Normaliza Unicode (corrige combinaciones mal codificadas) ---
	texto = unicodedata.normalize("NFKD", texto)

	# --- Elimina los diacríticos (tildes, diéresis, etc.) ---
	texto = "".join(c for c in texto if not unicodedata.combining(c))

	# --- Sustituye saltos de línea y espacios repetidos ---
	texto = re.sub(r"[\r\n\t]+", " ", texto)
	texto = re.sub(r"\s{2,}", " ", texto)

	# --- Elimina símbolos innecesarios ---
	texto = re.sub(r"[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑüÜ.,;:?!()¿¡+\-*/=<>\^% ]", "", texto)

	# --- Minúsculas uniformes ---
	texto = texto.lower().strip()

	return texto

In [5]:
# === 3. Segmentación ===
def segmentar_fijo(texto: str, tam_bloque: int = 800, solapamiento: int = 100) -> list:
	"""
		Divide el texto en bloques de longitud fija (en caracteres),
		con un solapamiento configurable para preservar contexto.
	"""
	chunks = []
	inicio = 0
	while inicio < len(texto):
		fin = inicio + tam_bloque
		fragmento = texto[inicio:fin]
		chunks.append(fragmento.strip())
		inicio += max(1, tam_bloque - solapamiento)
	return chunks


def segmentar_semantico(texto: str, tam_max: int = 800, solapamiento: int = 100) -> list:
	"""
		Divide el texto en fragmentos basados en oraciones (semántico).
		- Une oraciones hasta alcanzar tam_max.
		- Si una oración supera tam_max, se trocea con solapamiento.
	"""

	def _partir_largo(texto: str, tam_bloque: int, solapamiento: int) -> list:
		"""
			Parte un texto largo en ventanas de tamaño fijo con solapamiento.
		"""
		partes = []
		i = 0
		step = max(1, tam_bloque - solapamiento)
		while i < len(texto):
			partes.append(texto[i:i+tam_bloque])
			i += step
		return [p.strip() for p in partes if p.strip()]

	try:
		# Usar el modelo en español explícitamente para mejor segmentación de oraciones
		oraciones = sent_tokenize(texto, language="spanish")
	except LookupError:
		# Intentar descargar recursos en caliente si faltan
		nltk.download("punkt", quiet=True)
		nltk.download("punkt_tab", quiet=True)
		try:
			oraciones = sent_tokenize(texto, language="spanish")
		except LookupError:
			# Fallback simple si NLTK sigue fallando
			oraciones = re.split(r"(?<=[\.!?¡¿])\s+", texto)

	chunks, bloque = [], ""
	for oracion in oraciones:
		if len(oracion) > tam_max:

			# Antes de añadir esta oración muy larga, vaciar el bloque actual
			if bloque:
				chunks.append(bloque.strip())
				bloque = ""

			# Trocear la oración larga en ventanas con solapamiento
			trozos = _partir_largo(oracion, tam_bloque=tam_max, solapamiento=solapamiento)
			chunks.extend(trozos)
			continue

		# Acumular oraciones normales hasta tam_max
		if len(bloque) + len(oracion) + 1 <= tam_max:
			bloque = (bloque + " " + oracion).strip()
			
		else:
			if bloque:
				chunks.append(bloque.strip())
			bloque = oracion

	if bloque:
		chunks.append(bloque.strip())

	return chunks

### Aplicar el Preprocesamiento

In [6]:
# === Procesamiento general de PDFs ===
def preprocesar_archivos():
	metadata_records = []

	for archivo in os.listdir(carpeta_pdf):
		if archivo.endswith(".pdf"):
			ruta_pdf = os.path.join(carpeta_pdf, archivo)
			print(f"Procesando: {archivo}")

			# --- Extraer metadata desde el nombre ---
			partes = archivo.replace(".pdf", "").split("_")
			semana = partes[0] if len(partes) > 0 else "N/A"
			fecha = partes[3] if len(partes) > 3 else "00000000"
			version = partes[4] if len(partes) > 4 else "1"

			# --- Extraer y limpiar texto ---
			texto = extraer_texto_pdf(ruta_pdf)
			texto_limpio = limpiar_texto(texto)

			# --- Guardar texto limpio completo ---
			nombre_txt = archivo.replace(".pdf", ".txt")
			ruta_txt = os.path.join(carpeta_processed, nombre_txt)
			with open(ruta_txt, "w", encoding="utf-8") as f:
				f.write(texto_limpio)

			# --- A. Segmentar Fijo ---
			chunks_fijo = segmentar_fijo(texto_limpio, tam_bloque=800, solapamiento=100)
			for i, ch in enumerate(chunks_fijo):
				ruta_chunk = os.path.join(carpeta_chunks_fijo, f"{nombre_txt}_chunk_{i}.txt")
				with open(ruta_chunk, "w", encoding="utf-8") as f:
					f.write(ch)

			# --- B. Segmentar Semántico ---
			chunks_sem = segmentar_semantico(texto_limpio, tam_max=800, solapamiento=100)
			for i, ch in enumerate(chunks_sem):
				ruta_chunk = os.path.join(carpeta_chunks_sem, f"{nombre_txt}_chunk_{i}.txt")
				with open(ruta_chunk, "w", encoding="utf-8") as f:
					f.write(ch)

			# --- Registrar metadata ---
			metadata_records.append({
				"archivo_pdf": archivo,
				"archivo_txt": nombre_txt,
				"semana": semana,
				"fecha": fecha,
				"version": version,
				"ruta_txt": ruta_txt,
				"num_chunks_fijo": len(chunks_fijo),
				"num_chunks_sem": len(chunks_sem)
			})

			print(f"Texto limpio guardado y segmentado ({len(chunks_fijo)} + {len(chunks_sem)} chunks)")

	# === Guardar metadata general ===
	metadata_df = pd.DataFrame(metadata_records)
	metadata_path = f"{carpeta_data}/metadata.csv"
	metadata_df.to_csv(metadata_path, index=False, encoding="utf-8")
	print(f"\nMetadata registrada en: {metadata_path}")

In [ ]:
# # === Aplicar el preprocesamiento ===
# preprocesar_archivos()

Procesando: 10_SEMANA_AI_20251007_1-222887296.pdf
Texto limpio guardado y segmentado (11 + 11 chunks)
Procesando: 10_SEMANA_AI_20251007_1.pdf
Texto limpio guardado y segmentado (19 + 20 chunks)
Procesando: 10_SEMANA_AI_20251009_1.pdf
Texto limpio guardado y segmentado (11 + 11 chunks)
Procesando: 11_Semana_AI_20251014_1.pdf
Texto limpio guardado y segmentado (20 + 19 chunks)
Procesando: 11_Semana_AI_20251014_2.pdf
Texto limpio guardado y segmentado (24 + 24 chunks)
Procesando: 11_Semana_AI_20251014_3.pdf
Texto limpio guardado y segmentado (14 + 13 chunks)
Procesando: 11_SEMANA_AI_20251016_2.pdf
Texto limpio guardado y segmentado (12 + 12 chunks)
Procesando: 11_Semana_AI_20251016_4.pdf
Texto limpio guardado y segmentado (18 + 17 chunks)
Procesando: 12_SEMANA_AI_20251021_1.pdf
Texto limpio guardado y segmentado (11 + 11 chunks)
Procesando: 12_Semana_AI_20251021_2.pdf
Texto limpio guardado y segmentado (23 + 23 chunks)
Procesando: 12_SEMANA_AI_20251021_3.pdf
Texto limpio guardado y segmen

## 3. Tokenización y Embeddings

El objetivo de esta etapa es transformar cada documento procesado en una representación numérica (vector) que capture su significado semántico.
Estos embeddings permitirán que el agente RAG busque los fragmentos más relevantes según las preguntas del usuario.
  - Costo bajo y buena precisión.
  - Compatible con langchain y el cliente oficial de OpenAI.
- Entrada: textos limpios (uno por cada PDF procesado).
- Salida: vectores almacenados junto con su metadata en un archivo CSV.

En esta etapa se generan embeddings para los dos conjuntos de fragmentos (segmentación fija y segmentación semántica), utilizando el modelo local de Hugging Face ```intfloat/multilingual-e5-base```.

Este modelo convierte texto en vectores numéricos (768 dims) que capturan su significado semántico, permitiendo búsquedas por similitud. No requiere API ni conexión externa.

In [8]:
# === Configuración del modelo ===
EMBED_MODEL_NAME = "intfloat/multilingual-e5-base"
if "model" not in st.session_state:
    st.session_state.model = SentenceTransformer(EMBED_MODEL_NAME, device=device)
model = st.session_state.model

# === Función para procesar fragmentos ===
def procesar_directorio_chunks(carpeta_chunks: str, tipo: str, carpeta_salida: str) -> pd.DataFrame:
	"""
		Genera embeddings para todos los fragmentos de texto en una carpeta.
		'tipo' indica la segmentación: 'fixed' o 'semantic'.
	"""
	registros = []
	if not os.path.isdir(carpeta_chunks):
		print(f"[Aviso] Carpeta no encontrada: {carpeta_chunks}")
		return pd.DataFrame([])

	archivos = sorted(os.listdir(carpeta_chunks))
	print(f"\nGenerando embeddings ({tipo}): {len(archivos)} fragmentos encontrados.")

	textos, rutas = [], []

	# --- Recolección de fragmentos ---
	for archivo in archivos:
		ruta = os.path.join(carpeta_chunks, archivo)
		with open(ruta, "r", encoding="utf-8") as f:
			texto = f.read().strip()
		if texto:
			textos.append(texto)
			rutas.append(ruta)

	if not textos:
		print(f"[Aviso] No se encontraron textos válidos en {carpeta_chunks}")
		return pd.DataFrame([])

	# --- Generación por lotes ---
	embeddings = model.encode(
		textos,
		show_progress_bar=True,
		batch_size=16,
		convert_to_numpy=True,
		normalize_embeddings=True
	)

	# --- Construcción del DataFrame ---
	for i, (emb, ruta) in enumerate(zip(embeddings, rutas)):
		registros.append({
			"fragmento_id": f"{tipo}_{i}",
			"ruta_fragmento": ruta,
			"tipo_segmentacion": tipo,
			"embedding": emb.tolist()
		})

	# --- Guardado del DataFrame ---
	df = pd.DataFrame(registros)
	ruta_final = f"{carpeta_salida}/{tipo}.csv"
	df.to_csv(ruta_final, index=False, encoding="utf-8")

	print(f"Embeddings ({tipo}) guardados en {ruta_final}")
	return df

2025-11-05 18:57:59.974 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:57:59.976 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-11-05 18:58:04.612 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:04.613 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:04.614 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
# # === Procesar ambas segmentaciones ===
# df_fixed = procesar_directorio_chunks(carpeta_chunks_fijo, "fixed", carpeta_embeddings)
# df_sem   = procesar_directorio_chunks(carpeta_chunks_sem, "semantic", carpeta_embeddings)

# print("\nResumen final:")
# print(f"Embeddings fijos generados: {len(df_fixed)}")
# print(f"Embeddings semánticos generados: {len(df_sem)}")


Generando embeddings (fixed): 720 fragmentos encontrados.


Batches: 100%|██████████| 45/45 [00:08<00:00,  5.55it/s]


Embeddings (fixed) guardados en data/embeddings/fixed.csv

Generando embeddings (semantic): 722 fragmentos encontrados.


Batches: 100%|██████████| 46/46 [00:07<00:00,  6.17it/s]


Embeddings (semantic) guardados en data/embeddings/semantic.csv

Resumen final:
Embeddings fijos generados: 720
Embeddings semánticos generados: 722


## 4. Herramientas

### Construcción y Carga de Bases Vectoriales

In [10]:
# === Embeddings function ===
if "embedding_fn" not in st.session_state:
    st.session_state.embedding_fn = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
embedding_fn = st.session_state.embedding_fn

# === Construir / cargar VectorStores (Chroma) por segmentación ===
def _build_or_load_chroma(persist_dir: str, csv_path: str):
	"""
		Carga (si existe) o crea (si no) una base vectorial Chroma
		reutilizando los embeddings precomputados guardados en CSV.
		Compatible con langchain==1.0.3
	"""

	# Intentar cargar base existente
	try:
		db = Chroma(persist_directory=persist_dir, embedding_function=embedding_fn)
		count = db._collection.count()
		if count > 0:
			print(f"Chroma cargada desde {persist_dir} ({count} vectores)")
			return db
	except Exception:
		pass

	# Crear desde CSV
	if not os.path.exists(csv_path):
		raise FileNotFoundError(f"No existe el CSV: {csv_path}")

	print(f"Construyendo Chroma desde embeddings: {csv_path}")
	df = pd.read_csv(csv_path)
	df["embedding"] = df["embedding"].apply(lambda x: np.array(json.loads(x)))
	embeddings = np.vstack(df["embedding"].to_numpy())
	
	textos = []
	metadatas = []
	ids = []
	for _, row in df.iterrows():
		ruta = row["ruta_fragmento"]
		try:
			with open(ruta, "r", encoding="utf-8") as f:
				texto = f.read().strip()
		except FileNotFoundError:
			texto = "[Fragmento no encontrado]"
		textos.append(texto)
		metadatas.append({
			"ruta_fragmento": ruta,
			"tipo_segmentacion": row["tipo_segmentacion"],
			"fragmento_id": row["fragmento_id"],
		})
		ids.append(row["fragmento_id"])

	db = Chroma(
		persist_directory=persist_dir,
		embedding_function=embedding_fn
	)

	db._collection.add(
		ids=ids,
		documents=textos,
		embeddings=embeddings.astype(np.float32).tolist(),
		metadatas=metadatas,
	)

	db.persist()
	print(f"Chroma creada en {persist_dir} (docs: {len(df)})")
	return db

2025-11-05 18:58:26.580 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\lfben\AppData\Local\Temp\ipykernel_30244\1962929923.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  st.session_state.embedding_fn = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
2025-11-05 18:58:29.783 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:29.784 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread

In [11]:
# === Crear/cargar ambas colecciones ===
if "chroma_fixed" not in st.session_state or "chroma_sem" not in st.session_state:
    st.session_state.chroma_fixed = _build_or_load_chroma(f"{carpeta_db}/chroma_fixed", f"{carpeta_embeddings}/fixed.csv")
    st.session_state.chroma_sem = _build_or_load_chroma(f"{carpeta_db}/chroma_semantic", f"{carpeta_embeddings}/semantic.csv")

chroma_fixed = st.session_state.chroma_fixed
chroma_sem = st.session_state.chroma_sem

2025-11-05 18:58:29.795 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\lfben\AppData\Local\Temp\ipykernel_30244\1962929923.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(persist_directory=persist_dir, embedding_function=embedding_fn)


Construyendo Chroma desde embeddings: data/embeddings/fixed.csv


C:\Users\lfben\AppData\Local\Temp\ipykernel_30244\1962929923.py:63: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
2025-11-05 18:58:31.624 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:31.625 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Chroma creada en data/databases/chroma_fixed (docs: 720)
Construyendo Chroma desde embeddings: data/embeddings/semantic.csv


2025-11-05 18:58:32.883 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:32.884 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:32.884 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:32.885 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Chroma creada en data/databases/chroma_semantic (docs: 722)


### Herramienta RAG

In [12]:
def obtener_chunk_0(nombre_archivo: str, carpeta_texto: str = carpeta_chunks_sem) -> str:
	"""
		Obtiene el contenido del primer chunk (chunk_0.txt) del mismo documento base.
	"""

	# Identificar el nombre base (antes del sufijo _chunk_X.txt)
	base_name = nombre_archivo.split("_chunk_")[0]
	chunk_cero = f"{base_name}_chunk_0.txt"

	# Construir la ruta completa
	ruta_chunk_0 = os.path.join(carpeta_texto, chunk_cero)

	# Leer el contenido si existe
	if os.path.exists(ruta_chunk_0):
		try:
			with open(ruta_chunk_0, "r", encoding="utf-8") as f:
				contenido = f.read().strip()
				return contenido
		except Exception as e:
			return ""
	else:
		return ""


# Cargar modelo liviano en español
nlp = spacy.load("es_core_news_sm")
def detectar_autor_ia(archivo: str) -> str:
	"""
		Usa spaCy para detectar entidades tipo PERSON en el texto.
	"""
	texto = obtener_chunk_0(archivo)
	doc = nlp(texto)
	personas = [ent.text for ent in doc.ents if ent.label_ == "PER"]
	if personas:
		return personas[0].title()
	return "Desconocido"


def rag_search(query: str, mode: str = "semantic", k: int = 4):
	"""
		Realiza una búsqueda semántica en la base vectorial local.

		Parámetros:
			query: consulta del usuario
			mode: 'fixed' (segmentación fija) | 'semantic' (segmentación semántica)
			k: cantidad de fragmentos a recuperar

		Retorna:
			textos (List[str]), metadatos (List[dict])
	"""
	store = chroma_sem if mode == "semantic" else chroma_fixed

	prefixed_query = f"query: {query.strip()}"
	results = store.similarity_search(prefixed_query, k=k)
	
	textos = [r.page_content for r in results]
	metadatos = [r.metadata for r in results]

	return textos, metadatos


def rag_tool(query: str, mode="semantic", k=4) -> str:
	"""
		Tool de recuperación local: devuelve fragmentos relevantes.
	"""
	textos, metas = rag_search(query, mode=mode, k=k)
	bloques = []

	for t, m in zip(textos, metas):

		src = os.path.basename(m.get("ruta_fragmento", ""))
		frag_id = m.get("fragmento_id", "")
		bloques.append(f"\n")

		autor = detectar_autor_ia(src)
		bloques.append(f"[Fuente: {src} | ID: {frag_id} | Autor: {autor}]\n{t}\n")
		
	return "".join(bloques)

#### Creación de la Tool

In [13]:
RAG_Fixed_Tool = Tool(
    name=f"{agent_name} RAG Tool - Fixed",
    description="Busca en la base vectorial de los apuntes con segmentación fija.",
    func=lambda q: rag_tool(q, mode="fixed"),
)

RAG_Sem_Tool = Tool(
    name=f"{agent_name} RAG Tool - Semantic",
    description="Busca en la base vectorial de los apuntes con segmentación semántica.",
    func=lambda q: rag_tool(q, mode="semantic"),
)

print("RAG Tools creadas.")

RAG Tools creadas.


### Herramienta WebSearch

In [14]:
def web_search_tool(query: str, serpapi_key: str, max_results: int = 10) -> str:
    """
		Búsqueda web avanzada con SerpAPI.
		Recibe la API key como parámetro (sin depender de variables de entorno).
    """
    try:
        search = SerpAPIWrapper(serpapi_api_key=serpapi_key)
        results = search.results(query)

        formatted = []
        for r in results.get("organic_results", [])[:max_results]:
            title = r.get("title", "Sin título")
            snippet = r.get("snippet", "").strip()
            link = r.get("link", "")
            source = r.get("source", "Desconocido")

            formatted.append(f"[{title}]({link}) — {snippet} _(Fuente: {source})_")

        if not formatted:
            return f"No se encontraron resultados relevantes para '{query}'."

        return "\n\n".join(formatted)

    except Exception as e:
        return f"[Error en búsqueda web] {e}"


#### Creación de la Tool

In [15]:
SERPAPI_KEY = "dde1f999814b7ea544a7a9c6a64718f74423912d64b1182f2381557ed384feb8"

WebSearch_Tool = Tool(
    name="WebSearch_Tool",
    description="Realiza una búsqueda web con SerpAPI y devuelve enlaces relevantes con contexto.",
    func=lambda q: web_search_tool(q, serpapi_key=SERPAPI_KEY)
)

print("WebSearch Tool creada.")

WebSearch Tool creada.


## 5. Perfil, Orquestación y Memoria del Agente LLM

In [16]:
class SimpleMemory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_message(self, message):
        self.messages.append(message)

    def clear(self):
        self.messages = []

# === Memoria ===
memory = SimpleMemory()

# === Perfil del agente ===
AGENT_PERSONA = f"""
Eres {agent_name}, un asistente académico especializado en el curso de Inteligencia Artificial del Instituto Tecnológico de Costa Rica.

Rol:
- Actúas como tutor técnico y conceptual, capaz de responder preguntas relacionadas con los temas de IA vistos en clase (búsqueda, lógica, planificación, aprendizaje supervisado y no supervisado, redes neuronales, etc.).
- Siempre que cites información, indícalo claramente entre corchetes, con el nombre del archivo o la fuente.

Estilo de comunicación:
- Claro, formal y pedagógico.
- Explica los conceptos con ejemplos breves cuando sea necesario.
- Evita repeticiones o información innecesaria.
- Siempre referencia tanto el nombre de las fuentes que te den en el contexto, como las referencias al buscar en la web.

Restricciones:
- No digas que no puedes hacer búsquedas web ni menciones herramientas.
- No repitas la instrucción de cómo obtuviste la información, simplemente usa el contexto disponible.
- No inventes información. Si no hay datos suficientes en el contexto, indícalo con claridad y sugiere una búsqueda web.

Instrucción general:
- Utiliza exclusivamente la información incluida en el bloque [Contexto obtenido de ...]. Si la pregunta no tiene nada que ver con el contexto, entonces no hagas referencia al contexto.
- Si el contexto contiene material de apuntes o resultados de búsqueda web, intégralos directamente en tu respuesta.
- Mantén un tono académico, pero conciso.
"""

# === LLM ===
llm = OllamaLLM(model="llama3.2", temperature=0.2)

In [17]:
# === Preguntar al Agente ===
def responder_usuario(
		pregunta: str,
		k: int = 5,
		verbose: bool = False,
		segmentacion="semantic",
		tools=None,
		agent_profile: str = "",
		llm=None,
		memory=None
	):	
	"""
		Decide automáticamente si usar RAG o WebSearch
		según el tipo de pregunta.
	"""
	pregunta_lower = pregunta.lower()

	# --- Selección automática de herramienta ---
	if any(palabra in pregunta_lower for palabra in ["web", "internet", "buscar en internet", "navegar", "búsqueda web", "investiga"]):
		contexto = tools[2].run(pregunta)
		fuente = "búsqueda web"
		instruccion_extra = "Este contexto te lo dio el tool de la búsqueda web. Usa la información del contexto que se te dio, es el resultado de la búsqueda web para responder, tú no tienes que buscar nada extra. Referencia las fuentes y escribe los enlaces al final de tu respuesta."

	elif segmentacion == "fixed":
		contexto = tools[1].run(pregunta)
		fuente = "apuntes (segmentación fija)"
		instruccion_extra = "Este contexto te lo dio el tool del RAG fijo que obtiene información de los apuntes. Responde solo usando la información de los apuntes. Referencia las fuentes al final de tu respuesta. Si el contexto dado no tiene nada que ver con la pregunta, no referencias, solo habla normalmente."

	elif segmentacion == "semantic":
		contexto = tools[0].run(pregunta)
		fuente = "apuntes (segmentación semántica)"
		instruccion_extra = "Este contexto te lo dio el tool del RAG semántico que obtiene información de los apuntes. Responde solo usando la información de los apuntes. Referencia las fuentes al final de tu respuesta. Si el contexto dado no tiene nada que ver con la pregunta, no referencias, solo habla normalmente."

	else:
		contexto = ""
		fuente = ""
		instruccion_extra = ""

	# --- Carga del historial de la conversación ---
	historial = ""
	for m in memory.messages[-k:]:  		# solo los últimos k turnos
		if isinstance(m, HumanMessage):
			historial += f"Usuario: {m.content}\n"
		elif isinstance(m, AIMessage):
			historial += f"Asistente: {m.content}\n"

	# --- Construcción del prompt completo ---
	prompt = f"""
{agent_profile}

[Historial reciente de conversación]
{historial}

[Contexto obtenido de {fuente}]
{contexto}

{instruccion_extra}

Pregunta del usuario: {pregunta}
Tu respuesta:
	"""

	if (verbose):
		print(f"{prompt}")

	# --- Invocar modelo local de Ollama ---
	respuesta = llm.invoke(prompt)

	# --- Guardar Memoria ---
	memory.add_message(HumanMessage(content=pregunta))
	memory.add_message(AIMessage(content=respuesta))

	return respuesta.strip()

In [18]:
# # === Ejemplo de uso ===
# tools = [RAG_Sem_Tool, RAG_Fixed_Tool, WebSearch_Tool]
# print("------------------------------------------------------------")
# pregunta = "Cuales son las fórmulas de precision y de accuracy?"
# respuesta = responder_usuario(pregunta, verbose=True, segmentacion="semantic", tools=tools, agent_profile = AGENT_PERSONA, llm=llm, memory=memory)
# print("Pregunta:\n", pregunta)
# print("Respuesta del agente:\n", respuesta)

# print("------------------------------------------------------------")
# pregunta = "Haz una búsqueda web sobre las matrices de confusion"
# respuesta = responder_usuario(pregunta, verbose=False, segmentacion="semantic", tools=tools, agent_profile = AGENT_PERSONA, llm=llm, memory=memory)
# print("Pregunta:\n", pregunta)
# print("Respuesta del agente:\n", respuesta)

# print("------------------------------------------------------------")
# pregunta = "Cual fue la primera fórmula que te pregunté?"
# respuesta = responder_usuario(pregunta, verbose=True, segmentacion="semantic", tools=tools, agent_profile = AGENT_PERSONA, llm=llm, memory=memory)
# print("Pregunta:\n", pregunta)
# print("Respuesta del agente:\n", respuesta)

# print("------------------------------------------------------------")
# pregunta = "Haz una búsqueda web sobre el perceptrón"
# respuesta = responder_usuario(pregunta, verbose=False, segmentacion="semantic", tools=tools, agent_profile = AGENT_PERSONA, llm=llm, memory=memory)
# print("Pregunta:\n", pregunta)
# print("Respuesta del agente:\n", respuesta)

# print("------------------------------------------------------------")
# pregunta = "¿Cómo se calcula la derivada de la función de pérdida?"
# respuesta = responder_usuario(pregunta, verbose=True, segmentacion="semantic", tools=tools, agent_profile = AGENT_PERSONA, llm=llm, memory=memory)
# print("Pregunta:\n", pregunta)
# print("Respuesta del agente:\n", respuesta)

## 6. Aplicación

Hay que exportar este notebook a un .py y correr en una terminal:
```bash
streamlit run notebook.py
```

In [19]:
def crear_interfaz_agente():
	st.set_page_config(page_title="Agente QWERTY", layout="wide")

	# === Inicialización persistente ===
	if "agente_inicializado" not in st.session_state:
		print("Inicializando el agente por primera vez...")

		# --- Crear objetos persistentes ---
		st.session_state.llm = OllamaLLM(model="llama3.2", temperature=0.2)
		st.session_state.memory = SimpleMemory()
		st.session_state.tools = [RAG_Sem_Tool, RAG_Fixed_Tool, WebSearch_Tool]
		st.session_state.AGENT_PERSONA = AGENT_PERSONA

		st.session_state.mensajes = []
		st.session_state.agente_inicializado = True
		print("Agente inicializado correctamente")

	# === Interfaz de usuario ===
	col1, col2 = st.columns([0.7, 0.3])
	with col1:
		st.title("QWERTY")
		st.caption("Haz preguntas sobre los apuntes del curso de Inteligencia Artificial (LLaMA 3.2)")
	with col2:
		segmentacion = st.selectbox(
			"Segmentación",
			["semantic", "fixed"],
			index=0,
			format_func=lambda x: "Semántica" if x == "semantic" else "Fija"
		)
		st.session_state.segmentacion = segmentacion

	# === Historial de conversación ===
	for msg in st.session_state.mensajes:
		st.chat_message(msg["role"]).markdown(msg["content"])

	# === Entrada de usuario ===
	pregunta = st.chat_input(f"Escribe tu pregunta ({segmentacion})...")

	if pregunta:
		st.chat_message("user").markdown(pregunta)
		st.session_state.mensajes.append({"role": "user", "content": pregunta})

		with st.spinner("Pensando..."):
			respuesta = responder_usuario(
				pregunta,
				segmentacion=segmentacion,
				verbose=True,
				k=10,
				llm=st.session_state.llm,
				memory=st.session_state.memory,
				tools=st.session_state.tools,
				agent_profile=st.session_state.AGENT_PERSONA
			)

		st.chat_message("assistant").markdown(respuesta)
		st.session_state.mensajes.append({"role": "assistant", "content": respuesta})


# === Ejecución directa ===
if __name__ == "__main__":
	print("Iniciando aplicación...")
	crear_interfaz_agente()

2025-11-05 18:58:34.075 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:34.075 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Iniciando aplicación...
Inicializando el agente por primera vez...


2025-11-05 18:58:34.710 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:34.712 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:34.712 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:34.712 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:34.714 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:34.714 WARNING streamlit.run

Agente inicializado correctamente


2025-11-05 18:58:35.856 
  command:

    streamlit run c:\Users\lfben\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-05 18:58:35.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:35.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:35.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:35.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:35.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:35.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 18:58:35.861 Thread 'MainThread': missing ScriptRunContext! This warnin